In [1]:
from classes.league import League
import psycopg2
import random
import time
import pandas as pd

### Construct PostgreSQL Insert Statements from League object

In [2]:
def export_sql(league):
    if not league.scrapeable():
        print("League not scrapeable... Please try another league")
        return
    
    # Build insert statement to league table
    league_attrs = ['league_id', 'league_name', 'year', 'draft_type', 'num_teams', 'scoring_type',\
                   'roster_size', 'num_starters', 'num_bench', 'ppr_pts', 'keepers']
    insert_league_dict = {k: vars(league)[k] for k in league_attrs}
    insert_league_list = []
    for attr in league_attrs:
        if insert_league_dict[attr] == None:
            insert_league_list.append("NULL")
        elif type(insert_league_dict[attr]) == str:
            insert_league_list.append("'" + insert_league_dict[attr] + "'")
        else:
            insert_league_list.append(str(insert_league_dict[attr]))

    # Roster breakdown
    conn = psycopg2.connect('dbname=draft_analysis user=scraper')
    cur = conn.cursor()
    cur.execute("SELECT * FROM Draft2018.FantasyLeague LIMIT 0;")
    colnames = [desc[0] for desc in cur.description]
    
    old_pos = []
    for col in colnames:
        if 'num_rostered_' in col:
            old_pos.append(col)
    
    new_pos = []
    for pos, pos_count in league.roster_breakdown.items():
        if ('num_rostered_' + pos).lower() not in colnames:
            new_pos.append(('num_rostered_' + pos).lower())
    
    if len(new_pos) != 0:
        # Need to add new column in SQL database
        alter_pos_str = "ALTER TABLE Draft2018.FantasyLeague ADD COLUMN {0} smallint DEFAULT 0;"
        for pos in new_pos:
            cur.execute(alter_pos_str.format(pos))
    
    for col in old_pos + new_pos:
        pos = col.replace('num_rostered_', '')
        if pos in league.roster_breakdown:
            insert_league_list.append(str(league.roster_breakdown[pos]))
        else:
            insert_league_list.append('0')
        
    value_str = ', '.join(insert_league_list)
    insert_league_str = ["INSERT INTO Draft2018.FantasyLeague VALUES (" + value_str + ");"]

    # Build insert statement to owner table
    owner_attrs = ['league_id', 'team_names', 'team_records', 'team_pf', 'team_pa']
    insert_owner_dict = {k: vars(league)[k] for k in owner_attrs}
    insert_owner_strs = []
    for owner_id in league.team_ids:
        insert_owner_list = []
        insert_owner_list.append(str(owner_id))
        for attr in owner_attrs:
            if insert_owner_dict[attr] == None:
                insert_owner_list.append("NULL")
            elif type(insert_owner_dict[attr]) == str:
                insert_owner_list.append("'" + insert_owner_dict[attr] + "'")
            elif type(insert_owner_dict[attr]) == dict:
                if type(insert_owner_dict[attr][owner_id]) == list:
                    insert_owner_list.append( str(insert_owner_dict[attr][owner_id][0]))
                    insert_owner_list.append(str(insert_owner_dict[attr][owner_id][1]))
                    insert_owner_list.append(str(insert_owner_dict[attr][owner_id][2]))
                elif type(insert_owner_dict[attr][owner_id]) == str:
                    insert_owner_list.append("'" + insert_owner_dict[attr][owner_id] + "'")
                else:
                    insert_owner_list.append(str(insert_owner_dict[attr][owner_id]))
            else:
                insert_owner_list.append(str(insert_owner_dict[attr]))
        value_str = ', '.join(insert_owner_list)
        insert_owner_strs.append("INSERT INTO Draft2018.FantasyOwner VALUES (" + value_str + ");")
    
    # Update player table and build insert statements for player table           
    cur.execute("SELECT player_id FROM Draft2018.Player;")
    player_query = cur.fetchall()
    existing_ids = [player[0] for player in player_query]
    
    insert_player_strs = []
    for player_id,player_name,player_pos, player_team in league.drafted_players:
        if player_id not in existing_ids:
            insert_player = "INSERT INTO Draft2018.Player VALUES ({0},'{1}','{2}','{3}')"
            insert_player = insert_player.format(player_id, player_name, player_pos, player_team)
            insert_player_strs.append(insert_player)
        
    # Build insert statements for draft pick table
    insert_draft_strs = []
    for owner_id, pick_number, round_number, player_id, keeper in league.draft_picks:
        insert_draft = "INSERT INTO Draft2018.DraftPick VALUES({0},{1},{2},{3},{4},{5});"
        insert_draft = insert_draft.format(league.league_id, owner_id, pick_number,\
                                               round_number, player_id, keeper)
        insert_draft_strs.append(insert_draft)

    conn.commit()  
    cur.close()
    conn.close()

    return (insert_league_str, insert_owner_strs, insert_player_strs, insert_draft_strs)

### Scrape Data into Postgres DB

In [3]:
def scrape_draft(league_id, year):
    conn = psycopg2.connect("dbname=draft_analysis user=scraper")
    cur = conn.cursor()
    
    # Scrape and INSERT league/owners
    lg = League(league_id, year)

    if not lg.scrapeable():
        cur.close()
        conn.close()
        return
    
    lg.scrape_all()
    insert_statements = export_sql(lg)
    
    for insert_list in insert_statements:
        for insert in insert_list:
            cur.execute(insert)

    conn.commit()
    cur.close()
    conn.close()

In [5]:
id_list = list(pd.read_table('data/10000ids.csv',header=None)[0])
id_list.remove(614576)

In [6]:
conn = psycopg2.connect("dbname=draft_analysis user=scraper")
cur = conn.cursor()
cur.execute("SELECT league_id FROM Draft2017.FantasyLeague;")
existing_leagues = [entry[0] for entry in cur.fetchall()]
cur.close()
conn.close()

for league_id in id_list:
    if league_id not in existing_leagues:
        try:
            scrape_draft(league_id, 2017)
        except:
            pass
        time.sleep(random.random()*5)